In [1]:
#!/usr/bin/env python
import snowflake.connector as sfc
import pandas as pd
import time
import boto3
import s3fs 
import os
from io import StringIO
from smart_open import smart_open

In [2]:
# input personal key
mykey = ""
# input personal secret key 
mysecretkey = "" 
s3 = boto3.resource(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id=mykey,
    aws_secret_access_key=mysecretkey
)

os.environ["AWS_DEFAULT_REGION"] = 'us-west-2'
os.environ["AWS_ACCESS_KEY_ID"] = mykey
os.environ["AWS_SECRET_ACCESS_KEY"] = mysecretkey

# Connect to DB

In [3]:
omni = sfc.connect(
    user='cs2021_user3',
    password='Cap2021Stone-3',
    account='paciolan',
    warehouse='INTERNAL_ANALYTICS_WH',#'capstone2021_wh',
    database='omni_raw',
    schema='FUND',
    role = 'CAPSTONE2021_ROLE'#'INTERNAL_INFOMART_ROLE'
)

# Extract data

## DIM_DONOR (MARLENE)

In [ ]:
# raw data from DIM_DONOR
cs = omni.cursor()
cur = cs.execute('''SELECT *
                    from dim_donor
                    where organization_id != 'SPORTS'
                    and organization_id != 'ARTSDEMO'
                    and organization_id != 'CLASS1'
                    and organization_id != 'CLASS3A'
                    and organization_id != 'CICD80'
                ''')

donor_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
csv_buffer = StringIO()
donor_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/dim_donors.csv').put(Body=csv_buffer.getvalue())

In [ ]:
# clean and reformat for future use
donor_df = donor_df.drop(['DIM_ACCOUNT_KEY','DONOR_TYPE','DONOR_STATUS','DONOR_MEMBERSHIPS','MEMBERSHIPS'],axis=1)
donor_df['DONOR_TYPE_NAME'] = donor_df['DONOR_TYPE_NAME'].str.strip('{\n }')
donor_df['DONOR_TYPE_NAME'] = donor_df['DONOR_TYPE_NAME'].str.replace('"en_US":','')
donor_df['DONOR_STATUS_NAME'] = donor_df['DONOR_STATUS_NAME'].str.strip('{\n }')
donor_df['DONOR_STATUS_NAME'] = donor_df['DONOR_STATUS_NAME'].str.replace('"en_US":','')
donor_df['CATEGORIES'] = donor_df['CATEGORIES'].str.strip('{\n }')
donor_df['CATEGORY_START_DATES'] = donor_df['CATEGORY_START_DATES'].str.replace("{}","None")
donor_df['CATEGORY_END_DATES'] = donor_df['CATEGORY_END_DATES'].str.replace("{}","None")

donor_df.rename(columns={'ORGANIZATION_ID':'org_id','ID':'account_id', 'IS_DELETED':'is_deleted',
                                    'DONOR_TYPE_NAME':'donor_type','DONOR_STATUS_NAME':'donor_status',
                                    'YEARS_OF_DONATING':'years_of_donating','CATEGORIES':'categories',
                                    'CATEGORY_START_DATES':'cat_start_dates','CATEGORY_END_DATES':'cat_end_dates',
                                    'SYS_STATUS':'sys_status'
                                    }, inplace=True)

In [ ]:
# save the cleaned version to s3
csv_buffer = StringIO()
donor_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_dim_donors.csv').put(Body=csv_buffer.getvalue())

In [43]:
#donor_df = pd.read_csv(smart_open('s3://uci-capstone-custfields/raw/cleaned_dim_donors.csv'))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
donor_df

,org_id,account_id,is_deleted,donor_type,donor_status,years_of_donating,categories,cat_start_dates,cat_end_dates,sys_status
0,NEBRASKA,634566,False,"""Individual""","""Prospect""",0.0,NaN,None,None,1.0
1,ARKANSAS,443476,False,"""Individual""","""Active""",0.0,NaN,None,None,1.0
2,ARKANSAS,443495,False,"""Individual""","""Active""",0.0,NaN,None,None,1.0
3,ARKANSAS,443579,False,"""Individual""","""Active""",0.0,NaN,None,None,1.0
4,ARKANSAS,443554,False,"""Individual""","""Active""",0.0,NaN,None,None,1.0
...,...,...,...,...,...,...,...,...,...,...
5259325,OKLAHOMA,204432,False,"""Individual""","""New""",NaN,NaN,NaN,NaN,1.0
5259326,OKLAHOMA,572635,False,"""Individual""","""New""",0.0,NaN,None,None,1.0
5259327,OKLAHOMA,161379,False,"""Individual""","""New""",NaN,NaN,NaN,NaN,1.0
5259328,OKLAHOMA,809988,False,"""Individual""","""New""",2.0,"""TFPP"": ""61""",None,None,1.0


## DIM_DONOR_MEMBERSHIP (MARLENE)

In [ ]:
# raw 
cs = omni.cursor()
cur = cs.execute('''SELECT *
                    from dim_donor_membership
                    where organization_id != 'SPORTS'
                    and organization_id != 'ARTSDEMO'
                    and organization_id != 'CLASS1'
                    and organization_id != 'CLASS3A'
                    and organization_id != 'CICD80'
                ''')

donor_membership_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
csv_buffer = StringIO()
donor_membership_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/dim_donor_membership.csv').put(Body=csv_buffer.getvalue())

In [ ]:
donor_membership_df = donor_membership_df.drop(['DIM_ACCOUNT_KEY','DONOR_MEMBERSHIPS'],axis=1)
donor_membership_df.rename(columns={'ORGANIZATION_ID':'org_id','ACCOUNT_ID':'account_id', 'MEMBERSHIP_ID':'membership_id',
                                    'MEMBERSHIP_DRIVE_YEAR':'membership_drive_year','PLEDGED_LEVEL_NAME':'pledged_level',
                                    'RECEIPT_LEVEL_NAME':'receipt_level','TOTAL_DONATED':'total_donated',
                                    'TOTAL_PLEDGED':'total_pledged','DONOR_SYS_STATUS':'sys_status'}, inplace=True)
donor_membership_df

In [ ]:
# save the cleaned version to s3
csv_buffer = StringIO()
donor_membership_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_dim_donor_membership.csv').put(Body=csv_buffer.getvalue())

In [5]:
#donor_df = pd.read_csv(smart_open('s3://uci-capstone-custfields/raw/cleaned_dim_donor_membership.csv'))

In [35]:
year_donation = donor_df[['org_id','membership_drive_year','total_donated','total_pledged']].groupby(['org_id','membership_drive_year']).agg(['sum','median','mean','var','min','max']).reset_index()
# year_donation['avg_donation'] = year_donation['total_donated'] / year_donation['num']
# year_donation['avg_pledged'] = year_donation['total_pledged'] / year_donation['num']


In [36]:
year_donation.columns = ["_".join(x) if x[0] not in ["org_id","year"] else x[0] for x in year_donation.columns.ravel()]
yea_donation = year_donation.rename(columns={'membership_drive_year_':'year'},inplace = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  """Entry point for launching an IPython kernel.


In [37]:
year_donation

,org_id,year,total_donated_sum,total_donated_median,total_donated_mean,total_donated_var,total_donated_min,total_donated_max,total_pledged_sum,total_pledged_median,total_pledged_mean,total_pledged_var,total_pledged_min,total_pledged_max
0,ARKANSAS,2010,13033240.17,500.000,1220.798068,8.650827e+06,0.0,100000.00,13123743.67,500.0,1229.275353,8.666446e+06,0.0,100000.00
1,ARKANSAS,2011,19467473.66,750.000,1522.680771,8.872070e+06,0.0,98197.12,19593068.94,750.0,1532.504415,8.881154e+06,50.0,98197.12
2,ARKANSAS,2012,20610486.11,780.000,1605.053042,8.973810e+06,0.0,100000.00,20723626.80,800.0,1613.863936,8.974859e+06,0.0,100000.00
3,ARKANSAS,2013,20433144.84,800.000,1706.459399,9.941690e+06,0.0,110000.00,20533073.84,825.0,1714.804897,9.945875e+06,0.0,110000.00
4,ARKANSAS,2014,20277188.02,750.000,1663.291610,1.037045e+07,0.0,110000.00,20340067.52,750.0,1668.449473,1.036967e+07,0.0,110000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,USC,2017,26952673.00,3000.000,4656.647028,4.964515e+07,0.0,103200.00,27371619.09,3000.0,4729.028868,4.953575e+07,0.0,103200.00
134,USC,2018,27091649.85,1000.000,3517.939209,4.300926e+07,0.0,103200.00,26847424.26,1000.0,3486.225719,4.253012e+07,0.0,103200.00
135,USC,2019,31243304.74,1000.000,4012.238955,1.115720e+08,0.0,230000.00,31420447.43,1000.0,4034.987470,1.116723e+08,0.0,230000.00
136,USC,2020,145292.11,224.995,1513.459479,2.067817e+07,0.0,40000.00,243593.32,200.0,2537.430417,1.183145e+08,0.0,100000.00


In [38]:
# save the cleaned version to s3
csv_buffer = StringIO()
year_donation.to_csv(csv_buffer,index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_year_donation.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'XF9EWMEF5DBZ5QHK',
  'HostId': 'ghiiu9qutYLhhM8+Vcd+sSQvFQbWWHJ3Dx3UCmKfcCzmxEsqwqChZIfyeUU+vIZn/EOkoYHyUUo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ghiiu9qutYLhhM8+Vcd+sSQvFQbWWHJ3Dx3UCmKfcCzmxEsqwqChZIfyeUU+vIZn/EOkoYHyUUo=',
   'x-amz-request-id': 'XF9EWMEF5DBZ5QHK',
   'date': 'Sun, 11 Apr 2021 03:22:57 GMT',
   'x-amz-version-id': 'NzobXhOWJ_RifbRRuSgOJm3MUZIiJYjM',
   'etag': '"7829aac63b51cfae459ff7e5646948cd"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"7829aac63b51cfae459ff7e5646948cd"',
 'VersionId': 'NzobXhOWJ_RifbRRuSgOJm3MUZIiJYjM'}

## AccountPriorityPointsProgramDetails (MARLENE)

In [ ]:
cs = omni.cursor()
cur = cs.execute('''select DISTINCT DATA:acctid::string as account_id,
                           DATA:actualpoints::string as actual_points,
                           DATA:dbid::string as org_id,
                           DATA:pointscategoriesbreakdown as points_catg,
                           DATA:potentialpoints::int as potential_points,
                           DATA:progid::string as prog_id,
                           DATA:seq::int as seq
                    from ACCOUNTPRIORITYPOINTSPROGRAMDETAILS''')

priority_prog_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
# save the raw version
csv_buffer = StringIO()
priority_prog_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/priority_program.csv').put(Body=csv_buffer.getvalue())

In [4]:
priority_prog_df = pd.read_csv(smart_open('s3://uci-capstone-custfields/raw/priority_program.csv'))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# clean json format
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('{\n    "actualpoints": ','')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace(',\n    "name": {\n      "en_US":',',')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n    },\n    "potentialpoints": 0\n  },\n  ',',')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n    },\n    "potentialpoints": 0\n  }\n','')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n ','')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [7]:
priority_prog_df.rename(columns={'ACCOUNT_ID':'account_id','ACTUAL_POINTS':'actual_points',
                                'ORG_ID':'org_id','POINTS_CATG':'points_catg','POTENTIAL_POINTS':'potential_points',
                                'PROG_ID':'prog_id','SEQ':'seq'},inplace=True)
priority_prog_df = priority_prog_df.drop(['Unnamed: 0'],axis=1)
priority_prog_df

,account_id,actual_points,org_id,points_catg,potential_points,prog_id,seq
0,14926389.0,72502000,TAM,"{ ""1"": 45302000, ""Total Giving"",""10"": 0, ""Texa...",0.0,TAMA,2.0
1,20212.0,23031600,TAM,"{ ""1"": 12181600, ""Total Giving"",""10"": 0, ""Texa...",0.0,TAMA,6.0
2,20333.0,14601680,TAM,"{ ""1"": 3151680, ""Total Giving"",""10"": 0, ""Texas...",0.0,TAMA,13.0
3,602584.0,15729000,TAM,"{ ""1"": 9479000, ""Total Giving"",""10"": 0, ""Texas...",0.0,TAMA,18.0
4,20696.0,93336000,TAM,"{ ""1"": 72036000, ""Total Giving"",""10"": 0, ""Texa...",0.0,TAMA,19.0
...,...,...,...,...,...,...,...
770548,565129.0,220000,OKLAHOMA,"{ ""1"": 20000, ""Current Seating Gifts"",""10"": 0,...",0.0,CP,312.0
770549,579356.0,500000,OKLAHOMA,"{ ""1"": 100000, ""Current Seating Points"",""10"": ...",0.0,SP,319.0
770550,273945.0,69680000,OKLAHOMA,"{ ""1"": 1500000, ""Current Seating Points"",""10"":...",0.0,SP,322.0
770551,807691.0,1215000,OKLAHOMA,"{ ""1"": 0, ""Current Seating Points"",""10"": 0, ""S...",0.0,SP,331.0


In [8]:
#  save the cleaned version to s3
csv_buffer = StringIO()
priority_prog_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_priority_program.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '8J8THQZ91RJ2GGCR',
  'HostId': 'hGmstjMWvlxQzoOp2aVRtqKWd4ZRJp5ccF/pWO3umYZCOc18t4HOZ7B+HFuTo+muyBbzZ3b0ONk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hGmstjMWvlxQzoOp2aVRtqKWd4ZRJp5ccF/pWO3umYZCOc18t4HOZ7B+HFuTo+muyBbzZ3b0ONk=',
   'x-amz-request-id': '8J8THQZ91RJ2GGCR',
   'date': 'Fri, 09 Apr 2021 08:57:17 GMT',
   'x-amz-version-id': 'yaRGL_87YdVrVG_eHpc.FN.JvuNMxPxX',
   'etag': '"0e59989f8f8e439b908e949d2f3f8763"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0e59989f8f8e439b908e949d2f3f8763"',
 'VersionId': 'yaRGL_87YdVrVG_eHpc.FN.JvuNMxPxX'}

# Close Connection

In [ ]:
cs.close()
omni.close()